In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from split import *
from score import *
#from scipy.stats import chi2
from scipy import interpolate
import time 
%matplotlib inline 
# When I was implementing the algorithm Search_by_Triplet I see that PANDAS can do data selections
# more easily. The  query method gives us this possibility. 
# Not have sense to implement the orderring of data and similar things. 
# This line of code is to delete warnings. Specifically to rtankwarning of polyfit 
import warnings
warnings.simplefilter('ignore', np.RankWarning)

In [3]:
def rho(x,y):
    return np.sqrt(x*x + y*y)
def r(x,y,z):
    return np.sqrt(x*x + y*y + z*z)
def theta(x,y,z):
    return np.arccos(z/r(x,y,z))
def phi(x,y):
    return np.arctan(y/x)
def module(r):
    return np.sqrt(np.sum(r*r))
def r_e(z, r_l, r_c):
    #z   is a scalar. It is the position where we want to extrapolate. 
    #r_l is an array
    #r_c is an another array 
    z_c = r_c[2] 
    r_versor = (r_l - r_c)/module(r_l - r_c)               # computing r_versor
    r_versor_dot_z_versor = r_versor[2]  
    return r_c - r_versor/r_versor_dot_z_versor*(z_c - z)  # IMPORTANT WITH THE MINUS SIGN.

In [4]:
#df['theta'] = df.apply(lambda row: theta(row['x'], row['y'], row['z'] ), axis=1)
#df['phi']   = df.apply(lambda row: phi  (row['x'], row['y']),   axis=1)

In [4]:
def reading_data(fraction):
    name = 'data/RAMPData55microns50psInner200microns50psOuter_train.txt' # To be modified for others. 
    data_fraction = fraction

    df = pd.DataFrame()
    df = pd.read_csv(name, sep=' ') # All data.
    df,_ = Split_frac(df, data_fraction)
    return df

In [6]:
# DATAFRAME will be a global data. 
fraction = 0.004 
dphi     = 0.01         #this information is a parameter that have to be evaluated as a function of time. 

df_original = reading_data(fraction)  # where data is unmodified.
df          = df_original 
df_search   = df_original 

In [7]:
def sortbyphi(df):
    '''Description:
    Sort each D_i increasingly accourding to phi
    '''
    z = np.sort(df['z'].unique())
    df['phi'] = np.arctan(df['x']/df['y']) 
    modules = [] 
    for layer_i in z[::-1] :
        tmp = df.query(f'z=={layer_i}')
        #tmp['phi'] = tmp.apply(lambda row: phi(row['x'], row['y']), axis=1)
        
        tmp = tmp.sort_values('phi', ascending=True)
        modules.append(tmp)
    return modules 

In [8]:
def findcandidatewindows(left_mod, mod, right_mod, dphi):
    '''Description: 
        Compute the first and last candidates(the window) according to acceptance range(dphi) for each hit.
        SUPPOSSING THAT ALL DATA ARE ORDERED ACCOURDING TO PHI. THIS PROCCESS WAS DONE Previously
        In case of add more information to the modules, one easily can add throught the iteration 
    '''
    # CONVETION:     
    # l_m m  r_m         
    # |   |   |             
    # |   |   |    phi max  
    # |   |   |    phi      
    # |   |   |    phi min 
    # |   |   |             
    # The module in question is the one in the center
    #left_mod, mod, right_mod = modules 
    #dphi = acceptance_range  
    
    # This algorithm no develop a binary search. After, an intersting enhacement woudl be develop a function
    # in shuch a way that return the values of the condition
    
    #tmp = [] # Here I put the hit values related to right_hit_max, right_hit_min, or left_hit_max, left_hit_min
    
    #condition = f' {phi} - {dphi} < phi < {phi} - {dphi}' # condition
    #tmp = mod.query(condition)                            # defining the possibles hits
    #tmp['hit_id'][tmp.index[0]]       # the first hit   
    #tmp['hit_id'][tmp.index[-1]]      # the last  hit                                                    

    
    right_hit_max = [] 
    right_hit_min = [] 
    # ITERATION OVER PHI for RIGHT                     # ???????????????????? Here there are problems if not found values.
    for phi_i in mod['phi']:  
        # GET HIT 
        left      = phi_i - dphi
        right     = phi_i + dphi
        condition = f'{left} <= phi <= {right}'
        tmp_df    = right_mod.query(condition)
        #Getting the minimum and maximum

        m = 1 #tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        M = 1000 #tmp_df['hit_id'][tmp_df.index[-1]]   # maximum hit
        right_hit_min.append(m)
        right_hit_max.append(M)

    left_hit_max = [] 
    left_hit_min = [] 
    # ITERATION OVER PHI FOR LEFT
    for phi_i in mod['phi']:  
        # GET HIT 
        left      = phi_i - dphi
        right     = phi_i + dphi                                                                                                             
        condition = f'{left} <= phi <= {right}'
        tmp_df = right_mod.query(condition)
        #Getting the minimum and maximum
        m = 1 #tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        M = 1000 
        #m = tmp_df['hit_id'][tmp_df.index[0]]    # minumum hit
        #M = tmp_df['hit_id'][tmp_df.index[-1]]   # maximum hit
        left_hit_min.append(m)
        left_hit_max.append(M)
    
    # At the end of the algorithm we add the values of the left and right modules

    mod["right_hit_max"] = right_hit_max
    mod["right_hit_min"] = right_hit_min
    
    mod["left_hit_max"]  = left_hit_max
    mod["left_hit_min"]  = left_hit_min
    
    # With the help of two modules(left and riht)
    return mod

In [9]:
def trackseeding(mod, left_mod, right_mod):
    
    # mod_i are dataframes.
    # mod have the number of hits of max and min phi values. 
    # mod[] mod[]
    
    ''' 
    mod["right_hit_max"] = right_hit_max
    mod["right_hit_min"] = right_hit_min
    
    mod["left_hit_max"]  = left_hit_max
    mod["left_hit_min"]  = left_hit_min
    ''' 
    
    '''Description: 
        Checks the preceding and following modules for compatible hits using the above results.
        
        All triplets in the search window are fitted and compared.
        
        and the best one is kept as a track seed.
        
        stores its best found triplet
        Finding triplets is ap- plied in first instance to the modules
        that are further apart from the collision point
        Each triplet is the seed of a forming track
    '''
    #Necessary functions.
    def fit(triplet):
        phi_data = [ df.query(f'hit_id == {hit}')['phi'] for hit in triplet ]
        z_data   = [ df.query(f'hit_id == {hit}')['z'] for hit in triplet ]
        phi_data = [ hit.values[0] for hit in phi_data ]
        z_data   = [ hit.values[0] for hit in z_data ]


        #print(phi_data)
        #print(z_data)

        #plt.plot(phi_data, z_data)

        # Kind of fit: Linear
        fitting = np.polyfit(phi_data, z_data, 1)

        chiSquared = np.sum((np.polyval(fitting, z_data) - phi_data)**2) 
        return chiSquared 
    
       
    df_triplets = []
    for part in mod.iterrows():
        part = (part[1])  # IMPORTANT
        
        # hit_id
        # This process is neccessary 
        r_hit_max, r_hit_min = part["right_hit_max"], part["right_hit_min"]  
        l_hit_max, l_hit_min = part["left_hit_max"],  part["left_hit_min" ]  
        
        r_phi_max = right_mod.query(f"hit_id == {r_hit_max}")
        r_phi_min = right_mod.query(f"hit_id == {r_hit_min}")
        
        l_phi_max = left_mod.query(f"hit_id == {l_hit_max}")  
        l_phi_min = left_mod.query(f"hit_id == {l_hit_min}")  
        
        #Forming all Triplets. 

        tmp_left = df.query(f" {l_hit_min} <= phi <= {l_hit_max}")
               
        for L in tmp_left['phi'].values: 
            tmp_right = df.query(f" {r_hit_min} <= phi <= {r_hit_max}")
            for R in tmp_right['phi'].values:
                
        
                # All information it is found on the hit values.
                hit_center = int(part["hit_id"])             
                hit_left   = int(tmp_left.query(f" phi == {L}")['hit_id'].values)       
                hit_right  = int(tmp_right.query(f" phi == {R}")['hit_id'].values)                          
                
                # With this data we have built the triplets. 
                triplets = [hit_left, hit_center, hit_right]
                # This a lost of memory. I mean that call by hits and not by values is a lost 
                # of memory.
                chi2 = fit(triplets)
                # Finally we append the values of the data to a df_triplets
                
                #print("####?###")
                #print(triplets)
                #print(chi2)
                #print("#######")
                df_triplets.append(list(triplets)+[chi2])
    df_triplets = pd.DataFrame(df_triplets, columns = ['lef_hit', 'hit', 'right_hit', 'chi2']) 
    #        print("=========")
    #        print(df_triplets)
    #        print("=========")    
    # Up to this point it is necessary to have the values of df_triplets complete
    # Then the algorithm should continue to get the best choices according to the values
    # of chi2.  
              
    def best_choice(df_triplets):
        tracks = []
        for hit_c in df_triplets['hit'].values :
            #print(hit_c)
            tmp = df_triplets.query(f'hit == {hit_c}')
            #print(tmp)
            m = (tmp['chi2']).idxmin()
            #i = chi2values.idxmin() # Here we choice the values.
            #tracks.append((tmp.loc[i]).values) 
            t = (tmp.loc[m]).values
            #print(t)
            #print(t[:3])

            tracks.insert(0, list(t[:3]))
        return tracks
    
    tracks = best_choice(df_triplets)
    return tracks

In [10]:
def track_forwarding(tracks, work_module, dphi):
    new_tracks = [] 
    
    #Notation:
    # x0, y0, z0 is the EXTRAPOLATED track.               
    # X,  Y,  Z  is the last track on previous module.   
    # x,  y,  z  is the tracks on a window.                                                                 
    #Searching tracks on phi_e - dphi < phi < d that minimize the extrapolated function.
    #r0 = np.array([x0, y0, z0] )
    #r  = np.array([x, y, 1] )
    #R  = np.array([X,  Y,  Z ] )
    def mod(r):
        return np.sqrt(np.sum(r*r))
    def ext_func(r0, r1, r):
        # r0, r1, r are arrays
        dx2_plus_dy2 = mod(r0-r)  # distance between hits on the working module.
        dz2          = mod(r1-r0) # distance between the last two modules.                              
        return dx2_plus_dy2/dz2   
    
    
    z_e = float(work_module['z'].unique()) #z_position of work_module
    #phi_e
    #x_e
    #y_e
    #z_e
    
    # here the track is exactly the seed. Only for the 1th iteration
    
    for track in tracks: 
        data = []    
        vector_data = []
        for hit in track[0:2] : # Iteration over the 2 last modules to a module extrapolation.    
                                # IMPORTANT. I AM SUPPOSING THAT THE HITS ARE in separate modules.
            data.append(tuple((df.query(f'hit_id == {hit}')[['phi', 'z']]).values[0]))     
            vector_data.append(tuple((df.query(f'hit_id == {hit}')[['x', 'y', 'z']]).values[0]))  
        
        phi_data, z_data = zip(*data)
        #print(vector_data)
        #EXTRAPOLATED SEGMENT FUNCTION  
        ext_seg = interpolate.interp1d(z_data, phi_data, fill_value = "extrapolate" )
        phi_e = ext_seg(z_e)
        
        r_l, r_c = vector_data 
        r_l, r_c = np.array(r_l), np.array(r_c)
        
        x_e, y_e, z_e = r_e(z_e, r_l, r_c) # COMPUTING THE VALUES ON THE WORKING MODULE. It is good.
        
        #PROOF, HERE THE VALUE OF Z_E SHOULD BE THE SAME ON BOTH ********** IMPORTANT 
        
        #depend on z_e
        #y_e = #depend on z_e      
        
        #Open a Window centered on phi_e: 
        
        phi_a = phi_e - dphi
        phi_b = phi_e + dphi 
        
        df_work_module_window = work_module.query(f" {phi_a} <= phi <= {phi_b} " ) # I do not know how much time this line take 
        
        #This definition will be done after the loop.
        #df_candidates = pd.DataFrame(columns=["hit_id", "ext_fun"]) # This dataframe have to be reviwed
        
        
        hit_left = track[0]   
        R  = df.query(f'hit_id == {hit_left}')[['x','y','z']].values[0] # this value would have to change on each track
        r0 = np.array([x_e, y_e ,z_e ])
        
        
        print(R, "this is R")
        print(r0, "this is r0")
        tmp_candidates = []
        for index, row in df.iterrows(): # here the index is not important
            # Here I only need to have the information of position.
            r      =  row[['x', 'y', 'z']].values # this value is variable on each row.
            hit_id =  row['hit_id']    
            
            print("PROOF")
            print(r, hit_id)
            #Then I compute the value 
            ext_func_value = ext_func(r0, R, r)
            tmp_candidates.append( [hit_id, ext_func_value] )
            print(tmp_candidates)
        
        if tmp_candidates == []:                                            
            print("an error ocurred: with tmp_candidates is empty ")                                          
            break                                                                                              
        
        print("=====")
        print(tmp_candidates)
        print("=====")
        
        df_candidates = pd.DataFrame(tmp_candidates, columns=["hit_id", "ext_fun"])
        #if 
        
        # Finally 
        
        # I will find the minimum value of the extrapolation function 
        print("df_candidadtes ")
        print(df_candidates['ext_fun'])
        print("df_candidadtes")    
        index      = df_candidates['ext_fun'].idxmin()   
        new_hit_id = df_candidates['hit_id'][index]    
        new_track  = [new_hit_id] + track                            
        new_tracks.append(new_track)        
        # 
    return new_tracks # 

In [11]:
def GENERAL_ALGORITHM():
    global tracks      
    fraction = 0.004    
    dphi     = 0.01       
    
    df_original = reading_data(fraction)  # where data is unmodified.
    df_search   = df_original             # where I am searching 
    tracks = []                           # [[1,24, 5], [7,6,4] ,[346,7,32,], ... ]
    
    #SEPARATION BY MODULE
    modules = sortbyphi(df_original) 
    df = df_original                      # df is global variable 

    m = len(modules)                         # number of modules
    n = len(df_original['hit_id'].unique())  # number of hits                                             
       
    #ITERATION OVER MODULES ( ):
    for M_i in range(len(modules)-1, 2, -1):
        t1 = time.time()
        print(f"module number {M_i}")
        M_i = M_i - 1
        #1th STEP: module #m-1 = M_i
        left_mod  =  modules[M_i - 1]
        mod       =  modules[M_i    ]
        right_mod =  modules[M_i + 1]                                                                       
        
        print(" DOING  ... findcandidatewindows ")
        mod       = findcandidatewindows(left_mod, mod, right_mod, dphi)
        print(" ENDING ... findcandidatewindows ")
        
        
        
        #print("################################TRACKS###############################")
        #print(tracks) 
        #print("module #", M_i)
        tracks      = trackseeding(mod, left_mod, right_mod)
        work_module = modules[M_i - 2]
        tracks      = track_forwarding(tracks, work_module, dphi)  # return new tracks
        t2          = time.time()
        print("time per module", t2-t1) 
    return tracks 

In [12]:
def weaktrackfilter():
    '''Description: '''
    #               
    #                
    #

# Doing Proofs

In [8]:
fraction    = 0.004  
dphi        = 0.001                                                                                           

df_original = reading_data(fraction)     # where data is unmodified.
df_search   = df_original# where I am searching 
df          = df_original
tracks      =  []                              # [[1,24, 5], [7,6,4] ,[346,7,32,], ... ]

#SEPARATION BY MODULE                    # 
modules     = sortbyphi(df_original)         #                               
df          = df_original                         # 

m = len(modules)                         # number of modules
n = len(df_original['hit_id'].unique())  # number of hits    

#GENERAL_ALGORITHM()

NameError: name 'pd' is not defined